In [1]:
import os
import numpy as np
import pandas as pd
import mne
import matplotlib.pyplot as plt
from PIL import Image
from random import uniform
from IPython.utils import io

In [2]:
path_xlsx_file = 'dataEEG/'
file_name_xlsx = 'MGH_File_Annotations.xlsx'
annotation = pd.read_excel(path_xlsx_file+file_name_xlsx, sheet_name= 3)
annotation.head()

,Reader,Patient #,Recording,Recording Length [seconds],Recording Length (formatted),Task,Source,Created,Last Updated,Status,...,Epileptiform Discharges Focality Occipital,Recommend Further 10 20 Eeg,All Leads Working Throughout Recording,Quality Of Eeg,Recommend No Change To Patient Management Or Diagnosis,Epileptiform Discharges Focality Temporal,Is Eeg Usable For Clinical Purposes,Slowing Focality Temporal,Epileptiform Discharges Type,Slowing Type
0,Alice Lam,259,/mgh/smartphone/137_BAH.edf,1800.0,00:30:00,Check for Epileptiform Abnormalities (Smartphone),External,2018-06-29 04:00:00,Never,Completed,...,NaN,NaN,No,5,NaN,NaN,Yes,NaN,NaN,NaN
1,Alice Lam,105,/mgh/smartphone/134_ETT.edf,1706.0,00:28:26,Check for Epileptiform Abnormalities (Smartphone),External,2018-06-29 04:00:00,Never,Completed,...,NaN,NaN,Yes,8,NaN,NaN,Yes,NaN,NaN,NaN
2,Alice Lam,194,/mgh/xltek/67~ 67_ddd401b3-1cf8-483d-a289-cb20...,1366.0,00:22:46,Check for Epileptiform Abnormalities (XLTEK),Internal,2018-11-12 05:38:00,2018-11-17 05:05:00,Completed,...,NaN,NaN,No,6,NaN,NaN,Yes,NaN,NaN,NaN
3,Alice Lam,11,/mgh/xltek/66~ 66_3279d30c-9286-47e4-a9ca-693e...,1454.0,00:24:14,Check for Epileptiform Abnormalities (XLTEK),Internal,2018-11-12 05:38:00,2018-11-17 05:18:00,Completed,...,NaN,NaN,No,1,NaN,NaN,No,NaN,NaN,NaN
4,Alice Lam,260,/mgh/smartphone/138_CAM (2).edf,1073.0,00:17:53,Check for Epileptiform Abnormalities (Smartphone),External,2018-06-29 04:00:00,Never,Completed,...,NaN,NaN,No,5,NaN,NaN,Yes,NaN,NaN,NaN


In [3]:
# Only need to run this cell the first time you use the script

annotation_path = annotation['Recording']
annotation_Quality = annotation['Quality Of Eeg']

file_names = []
quality = []

for i in range(len(annotation_path)):
    if type(annotation_Quality[i]) == int:
        file_names.append(annotation_path[i].split('/')[-1])
        quality.append(int(int(annotation_Quality[i]) > 3))

In [4]:
path = 'dataEEG/'
fileName_quality = []

idx = None

for folder, subfolder, filenames in os.walk(path):
    for edf_file in filenames:
        
        idx = None
        
        #search for filename index
        for i, file_name in enumerate(file_names):
            if file_name == edf_file:
                idx = i
                break
        
        # create list of tuples contaning the patg to the fil and gender
        if idx != None:
            fileName_quality.append((folder+'/'+edf_file, quality[idx]))
        else:
            continue

In [8]:
###############################################################################################################################
###############################################################################################################################
###############################################################################################################################

################################# ONLY RUN THIS CELL ONCE OTHERWISE YOU DOUBLICATE THE IMAGES #################################

###############################################################################################################################
###############################################################################################################################
###############################################################################################################################

#breaks = 0
window_size = 30
count = 0
channel = 0

# Creating data
for path, q in fileName_quality:
    with io.capture_output() as captured:
        raw  = mne.io.read_raw_edf(path);
        freq = raw.info['sfreq'];
        num_samples = raw.n_times
    lower_bound = list(range(0, int(num_samples/freq)-window_size, window_size-int(window_size/2)))
    upper_bound = list(range(window_size, int(num_samples/freq), window_size-int(window_size/2)))
    
    
    for low, up in zip(lower_bound,upper_bound):
        # Copy the data so we stil have the full data file after crop
        with io.capture_output() as captured:
            raw_subset = raw.copy();
        
        # crop the copy
        with io.capture_output() as captured:
            raw_subset.crop(tmin = low, tmax = up).load_data();
        
        # filter the data
        with io.capture_output() as captured:
            raw_subset.filter(l_freq=0.4, h_freq=40);
    
        # extract the data values
        with io.capture_output() as captured:
            data = raw_subset.get_data();
        
        # Choose channel
        data = data[channel]
        
        # Create spectrogram
        pxx, freqs, bins, im = plt.specgram(data, Fs = freq, cmap = 'gray',)
        pxx = np.log1p(pxx)
        #plt.yscale('log')
        plt.axis('off')
        
        
        # decide between test or train
        test_or_train = uniform(0, 1)
        
        # save spectrogram into correct folder
        if q == 1: # Tjek if current data is recorded on male
            Img_name = '3_or_above'+str(count)+'_'+str(channel)+'.jpg'
            if test_or_train < 0.9:
                plt.savefig('D:/qulety_data/Train/3_or_above/'+Img_name, edgecolor = None, bbox_inches='tight',pad_inches = 0)
            else:
                plt.savefig('D:/qulety_data/Test/3_or_above/'+Img_name, edgecolor = None, bbox_inches='tight',pad_inches = 0)
        else:
            Img_name = 'below_3'+str(count)+'_'+str(channel)+'.jpg'
            if test_or_train < 0.9:
                plt.savefig('D:/qulety_data/Train/below_3/'+Img_name, edgecolor = None, bbox_inches='tight',pad_inches = 0)
            else:
                plt.savefig('D:/qulety_data/Test/below_3/'+Img_name, edgecolor = None, bbox_inches='tight',pad_inches = 0)
        del raw_subset
        plt.close()
        count +=1
    del raw
    #breaks +=1
    #if breaks > 0:
        #break
            

MemoryError: Unable to allocate array with shape (476800,) and data type float64